# Calculate Simple Cometary Magnitude

In [ ]:
import numpy as np
import pandas as pd
import astropy.units as u
from sorcha.lsstcomet import Comet
from sbpy.activity import Afrho
import synphot

from sorcha.modules.PPCalculateSimpleCometaryMagnitude import PPCalculateSimpleCometaryMagnitude

The lsstcomet code used by sorcha validates its results by comparing them to the cometary magnitude calculated by sbpy. We will do the same.

First, calculating using sbpy:

In [ ]:
g = {'rh': 2.0 * u.au, 'delta': 1.0 * u.au, 'phase': 0 * u.deg}
afrho = Afrho(100 * 2**-2, 'cm')
tab = np.loadtxt('lsst-total-r.dat').T
r = synphot.SpectralElement(synphot.Empirical1D, points=tab[0] * u.nm,
                            lookup_table=tab[1])
rap = 1 * u.arcsec
m0 = afrho.to_fluxd(r, rap, g, unit=u.ABmag).value

comet = Comet(R=1, afrho1=100, k=-2)
m = comet.mag(g, 'r', rap=rap.value, nucleus=False)

m_sbpy = -2.5 * np.log10(10 ** (-0.4 * m) + 10 ** (-0.4 * 7.3))

Now a test dataset must be created using the same values.

In [ ]:
test_dict = {'MJD': [2459215.5],
              'H_r': [7.3],
              'AstRange(km)': [1. * 1.495978707e8],
              'Ast-Sun(km)': [2. * 1.495978707e8],
              'Sun-Ast-Obs(deg)': [0],
              'afrho1': [100],
              'q':[1. * 1.495978707e8],
              'k':[-2],
              'optFilter':'r'}

test_data = pd.DataFrame(test_dict)

Calculating cometary apparent magnitude using the SSPP function and comparing:

In [ ]:
test_data = PPCalculateSimpleCometaryMagnitude(test_data.copy(), 'r', '')

In [ ]:
m_sspp = test_data['H_r']

In [ ]:
assert np.isclose(m_sspp, m_sbpy, atol=0.05)